In [1]:
# add utils to path
import sys
sys.path.append('../')

import json
import numpy as np

In [2]:
def shift_and_normalize_points(points, shift=[0., 0.5, 0.5, 0.5], scale=1):
    # Shift the points
    points_shifted = np.array(points) - np.array(shift)
    norm = np.linalg.norm(points_shifted)
    return (points_shifted / norm).tolist()
    

def unshift_and_normalize_points(points, shift=[0.0, 0.5, 0.5, 0.5], scale=1):
    # Unnormalize the points
    points_unshifted = np.array(points) + np.array(shift)
    norm = np.linalg.norm(points_unshifted)
    return (points_unshifted / norm).tolist()

def quaternion_multiply(q1, q2):
    # Unpack the quaternions
    w1, x1, y1, z1 = q1
    w2, x2, y2, z2 = q2
    
    # Perform the multiplication
    w = w1*w2 - x1*x2 - y1*y2 - z1*z2
    x = w1*x2 + x1*w2 + y1*z2 - z1*y2
    y = w1*y2 - x1*z2 + y1*w2 + z1*x2
    z = w1*z2 + x1*y2 - y1*x2 + z1*w2
    
    return [w, x, y, z]

# def rotate_around_z(R, T, theta, shift=[0.5, 0.5]):
#     # Convert theta from degrees to radians
#     theta_rad = np.radians(theta)
    
#     # Create a quaternion for rotation around the Z axis
#     q_rot_z = [np.cos(theta_rad / 2), 0, 0, np.sin(theta_rad / 2)]
#     q_rot_x = [np.cos(theta_rad / 2), np.sin(theta_rad / 2), 0, 0]
#     q_rot_y = [np.cos(theta_rad / 2), 0, np.sin(theta_rad / 2), 0]
#     q_rot = q_rot_y

#     T_shifted = [T[0] - shift[0], T[1] - shift[1], T[2]]
#     T_shifted_quat = [0] + T_shifted
#     T_rotated_shifted_quat = quaternion_multiply(
#         quaternion_multiply(q_rot, T_shifted_quat),
#         [q_rot[0], -q_rot[1], -q_rot[2], -q_rot[3]]
#     )

#     T_rotated = [
#         T_rotated_shifted_quat[1] + shift[0],
#         T_rotated_shifted_quat[2] + shift[1],
#         T_rotated_shifted_quat[3]
#     ]

#     # Rotate R
#     R_rotated = quaternion_multiply(q_rot, R)
    
    
#     return R_rotated, T_rotated
def normalize_quaternion(q):
    # make array if not already
    q = np.array(q)
    """Normalize a quaternion."""
    norm = np.linalg.norm(q)
    return (q / norm).tolist()

def quaternion_to_rotation_matrix(q):
    """Convert a quaternion into a 3x3 rotation matrix."""
    w, x, y, z = q
    R = np.array([
        [1 - 2*y*y - 2*z*z,     2*x*y - 2*z*w,     2*x*z + 2*y*w],
        [2*x*y + 2*z*w,     1 - 2*x*x - 2*z*z,     2*y*z - 2*x*w],
        [2*x*z - 2*y*w,     2*y*z + 2*x*w,     1 - 2*x*x - 2*y*y]
    ])
    return R

def rotate_around_y(R, T, theta, shift=[0.5, 0.5]):
    # Convert theta from degrees to radians
    theta_rad = np.radians(theta)
    
    # Create a quaternion for rotation around the Y axis when 
    # R is in form [w, x, y, z]
    q_rot_y = [np.cos(theta_rad / 2), 0, np.sin(theta_rad / 2), 0]
    q_rot = q_rot_y

    T_shifted = [T[0] - shift[0], T[1], T[2] - shift[1]]
    T_shifted_quat = [0] + T_shifted
    T_rotated_shifted_quat = quaternion_multiply(
        quaternion_multiply(q_rot, T_shifted_quat),
        [q_rot[0], -q_rot[1], -q_rot[2], -q_rot[3]]
    )

    T_rotated = [
        T_rotated_shifted_quat[1] + shift[0],
        T_rotated_shifted_quat[2],
        T_rotated_shifted_quat[3] + shift[1]
    ]

    # Rotate R
    R_shifted = normalize_quaternion([R[0], R[1] - shift[0], R[2], R[3] - shift[1]])
    R_rotated = quaternion_multiply(
        quaternion_multiply(q_rot, R_shifted),
        [q_rot[0], -q_rot[1], -q_rot[2], -q_rot[3]]
    )
    R_rotated = [
        R_rotated[0],
        R_rotated[1] + shift[0],
        R_rotated[2],
        R_rotated[3] + shift[1]
    ]

    return R_rotated, T_rotated

In [3]:
cam = json.load(open('../cam.json'))

In [4]:
cam

{'loop': False,
 'path': [{'R': [0.9380469918251038,
    -0.0429956391453743,
    0.3198738396167755,
    0.12608736753463745],
   'T': [-1.1177868843078613, 0.8019675016403198, 1.45546555519104],
   'aperture_size': 0.0,
   'fov': 50.625,
   'glow_mode': 0,
   'glow_y_cutoff': 0.0,
   'scale': 1.1269716024398804,
   'slice': 0.14271652698516846}],
 'time': 1.0}

In [5]:
R = cam['path'][0]['R']
T = cam['path'][0]['T']

In [6]:
T

[-1.1177868843078613, 0.8019675016403198, 1.45546555519104]

In [7]:
R_rotated, T_rotated = rotate_around_y(R, T, 90)

In [8]:
R_rotated

[0.7880504498761118,
 0.1858771247105826,
 0.2687250483295598,
 0.9561689994620881]

In [9]:
cam_rotated = cam.copy()
# cam_rotated['path'].append(cam['path'][0].copy())
# cam_rotated['path'][1]['R'] = R_rotated
# cam_rotated['path'][1]['T'] = T_rotated

In [10]:
# rotate by 90, 180, 270
# R_shifted = shift_and_normalize_points(R)
for i in range(0, 360, 10):
    R_rotated, T_rotated = rotate_around_y(R, T, i)
    cam_rotated['path'].append(cam['path'][0].copy())
    cam_rotated['path'][-1]['R'] = R_rotated # unshift_and_normalize_points(R_rotated)
    cam_rotated['path'][-1]['T'] = T_rotated

In [11]:
# save as cam_rotated.json
with open('../cam_rotated_2.json', 'w') as f:
    json.dump(cam_rotated, f, indent=4)